In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.setrecursionlimit(1000)

In [2]:
import pandas as pd
import numpy as np

from keras import Model
from keras.layers import Embedding, Input, Flatten, Dot, Add
from keras.optimizers import Adam
from keras.regularizers import l2
from sklearn.model_selection import train_test_split

# Train model

In [56]:
ratings.head()

,Unnamed: 0,user_id,movie_id,rating,Date
0,0,1,1,3,1997-12-04 15:55:49
1,1,2,2,3,1998-04-04 19:22:22
2,2,3,3,1,1997-11-07 07:18:36
3,3,4,4,2,1997-11-27 05:02:03
4,4,5,5,1,1998-02-02 05:33:16


In [63]:
ratings = pd.read_csv("Data/scores.csv")

# create a dictionary of indexes for the users and movies mapping every ID to a given index
u_unique = ratings.user_id.unique()
user2Idx = {o:i+1 for i,o in enumerate(u_unique)}

m_unique = ratings.movie_id.unique()
movie2Idx = {o:i+1 for i,o in enumerate(m_unique)}

# Replace the users and movies ids with the new index value
ratings["user_id"] = ratings["user_id"].apply(lambda x: user2Idx[x])
ratings["movie_id"] = ratings["movie_id"].apply(lambda x: movie2Idx[x])

n_users = int(ratings.user_id.nunique())
n_movies = int(ratings.movie_id.nunique())

n_users, n_movies

(943, 1682)

In [64]:
def build_keras_model(users_number, movies_number, latent_factors = 5, add_bias = False, loss="mean_squared_error", learning_rate=0.001, metrics=[keras.metrics.RootMeanSquaredError()]):
    user_input = Input(shape=[1],name="User")
    movie_input = Input(shape=[1],name="Item")
    
    # Movie embedding layer
    movie_embedding = Embedding(
        movies_number + 1,
        latent_factors,
        embeddings_regularizer=l2(0.001),
        name = "Movie-Embedding"
    )(movie_input)
    
    movie_vec = Flatten(name = "FlattenMovies")(movie_embedding)

    # User embedding layer
    user_embedding = Embedding(
        users_number + 1,
        latent_factors,
        #embeddings_regularizer=l2(0.001),
        name = "User-Embedding"
    )(user_input)
    
    user_vec = Flatten(name = "FlattenUsers")(user_embedding)
    
    prod = Dot(axes=1, name="DotProduct")([movie_vec,user_vec])
    
    if add_bias:
        # Movie bias Embedding layer
        movie_bias_embedding = Embedding(
            movies_number+1,
            1,
            embeddings_regularizer = l2(0.001),
            name = "Movie-Bias-Embedding"
        )(movie_input)
        
        movie_bias = Flatten(name="FlattenMoviesBias")(movie_bias_embedding)
        
        # User bias Embedding layer
        user_bias_embedding = Embedding(
            users_number+1,
            1,
            #embeddings_regularizer = l2(0.001),
            name = "User-Bias-Embedding"
        )(user_input)
        
        user_bias = Flatten(name="FlattenUserBias")(user_bias_embedding)
        
        prod = Add()([prod,user_bias,movie_bias])
        
    model = Model([user_input,movie_input],prod)
    model.compile(Adam(learning_rate = learning_rate), loss, metrics=metrics)
    return model

In [65]:
import keras
keras.metrics.RootMeanSquaredError()

In [66]:
# split dataset

eval_size = 0.2
ratings_train, ratings_val = train_test_split(ratings, test_size = eval_size)

best_lf = 0
best_mse = float('inf')
epochs = 100
latent_factor = 13
model = build_keras_model(n_users,n_movies,latent_factors = latent_factor, add_bias = True)
history = model.fit(
    [ratings_train.user_id, ratings_train.movie_id],
    ratings_train.rating,
    batch_size=320,
    validation_data = ([ratings_val.user_id, ratings_val.movie_id],ratings_val.rating),
    epochs = epochs,
    verbose = 1
)

metrics_train = model.evaluate([ratings_train.user_id,ratings_train.movie_id],ratings_train.rating)
metrics_val = model.evaluate([ratings_val.user_id,ratings_val.movie_id],ratings_val.rating)

print(f"RMSE: {metrics_val[1]} for latent_factors = {latent_factor}")

Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 0s 4us/step - loss: 12.7722 - root_mean_squared_error: 3.5720 - val_loss: 11.5678 - val_root_mean_squared_error: 3.3949
Epoch 2/100
80000/80000 [==============================] - 0s 3us/step - loss: 9.2158 - root_mean_squared_error: 2.9980 - val_loss: 6.4903 - val_root_mean_squared_error: 2.4484
Epoch 3/100
80000/80000 [==============================] - 0s 3us/step - loss: 4.7551 - root_mean_squared_error: 1.9954 - val_loss: 3.6765 - val_root_mean_squared_error: 1.6371
Epoch 4/100
80000/80000 [==============================] - 0s 3us/step - loss: 3.2544 - root_mean_squared_error: 1.4703 - val_loss: 3.0056 - val_root_mean_squared_error: 1.3637
Epoch 5/100
80000/80000 [==============================] - 0s 3us/step - loss: 2.8075 - root_mean_squared_error: 1.2885 - val_loss: 2.6993 - val_root_mean_squared_error: 1.2499
Epoch 6/100
80000/80000 [==============================] - 0s 3

In [51]:
import tensorflow as tf
tf.reset_default_graph()

In [ ]:
movies = pd.read_csv("Data/peliculas.csv")
users = pd.read_csv("Data/usuarios.csv")

,Unnamed: 0,user_id,movie_id,rating,Date
0,0,196,242,3,1997-12-04 15:55:49
1,1,186,302,3,1998-04-04 19:22:22
2,2,22,377,1,1997-11-07 07:18:36


In [14]:
users.head(3)

,id,Occupation,Active Since
0,1,technician,1997-09-22 21:57:58
1,2,other,1998-02-27 03:26:00
2,3,writer,1998-03-07 02:15:39


In [15]:
movies.head(3)

,id,Name,Release Date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [11]:
users["id"].apply(lambda x: user)

0        1
1        2
2        3
3        4
4        5
      ... 
938    939
939    940
940    941
941    942
942    943
Name: id, Length: 943, dtype: int64